In [1]:
from glob import glob

import numpy as np
import xarray as xr
import salem
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

from python import wind_utils as wu
from python import utils as pu

In [2]:
month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

In [4]:
import importlib
importlib.reload(wu)

<module 'python.wind_utils' from '/home/fs01/dcl257/projects/acorn-julia/src/python/wind_utils.py'>

In [ ]:
%%time
# Get TGW input solar data
climate_paths = np.array([glob(f"{pu.tgw_path}/historical_1980_2019/hourly/tgw_wrf_historical_hourly_{year}*.nc") 
                          for year in range(2007, 2014)]).flatten()

df = wu.prepare_wind_data(
    climate_paths = climate_paths,
    wind_vars = ['U10', 'V10'],
    lat_name="south_north",
    lon_name="west_east",
    curvilinear=True,
)

In [ ]:
# Add in climate data windspeed
df['ws'] = np.sqrt(df['U10']**2 + df['V10']**2)

In [ ]:
# Compute the stability coefficients
res = wu.get_stability_coefficients(df, 'ws', 'wtk_ws').reset_index()

In [ ]:
# Plot the stability coefficients
fig, axs = plt.subplots(3, 4, figsize=(10,8), layout='tight', sharey=True, sharex=True)
axs = axs.flatten()

for month in range(1,13):
    ax = axs[month-1]
    res[res['month'] == month].plot(x='hour', y='alpha', legend=False, ax=ax)
    ax.set_title(month_names[month-1])

plt.show()

In [ ]:
# Merge and plot results
df = pd.merge(df, res, on =['hour', 'month'])
df['ws_corrected'] = df['ws'] * (100/10)**df['alpha']

In [ ]:
latlons = df[['wtk_lat', 'wtk_lon']].value_counts().index.unique()

In [ ]:
# Plot the stability coefficients
fig, axs = plt.subplots(3, 4, figsize=(10,8), layout='tight', sharey=True, sharex=True)
axs = axs.flatten()

for month in range(1,13):
    ax = axs[month-1]
    df_tmp = df[df['month'] == month]
    
    df_tmp.plot.scatter(x='ws_corrected', y='wtk_ws', legend=False, ax=ax)

    # Add fit info
    r2 = (np.corrcoef(df_tmp.dropna()['ws_corrected'], df_tmp.dropna()['wtk_ws'])[0, 1]** 2)
    rmse = np.sqrt(np.mean((df_tmp.dropna()['ws_corrected'] - df_tmp.dropna()['wtk_ws']) ** 2))
    # Add 1:1 line
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", color="black")

    ax.set_title(f"{month_names[month-1]} (R$^2$: {r2:.2f}, RMSE: {rmse:.2f})")
    ax.grid(alpha=0.5)
    ax.set_xlabel("")
    ax.set_ylabel("")

fig.supylabel('WTK wind [m/s]')
fig.supxlabel('TGW wind [m/s]')

plt.show()